In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 27 13:28:27 2018
@author: PavitrakumarPC
"""

import numpy as np
import cv2
import os
import pandas as pd
import h5py


def construct_annotation_files(annotations):
    if not os.path.exists(r"data\VOCdevkit\VOC2007\Annotations"):
        os.makedirs(r"data\VOCdevkit\VOC2007\Annotations")

    for i in range(len(annotations['digitStruct']['bbox'])):
        image_name = get_name(i, annotations)
        img = cv2.imread("dataset/train/"+image_name)
        ann = get_bbox(i, annotations)

        with open(r"data\VOCdevkit\VOC2007\Annotations\{}.xml".format(image_name[:-4]), 'w') as file:
            file.write("<annotation>\n")
            file.write("  <filename>" + image_name + "</filename>\n")
            file.write("  <size>\n")
            file.write("    <width>" + str(img.shape[1]) + "</width>\n")
            file.write("    <height>" + str(img.shape[0]) + "</height>\n")
            file.write("    <depth>" + str(img.shape[2]) + "</depth>\n")
            file.write("  </size>\n")

            for j,name in enumerate(ann['label']):
                file.write("  <object>\n")
                file.write("    <name>{}</name>\n".format(int(name)))
                file.write("    <bndbox>\n")
                xmin = ann['left'][j]
                ymin = ann['top'][j]
                file.write("      <xmin>{}</xmin>\n".format(int(xmin)))
                file.write("      <ymin>{}</ymin>\n".format(int(ymin)))
                file.write("      <xmax>{}</xmax>\n".format(int(xmin + ann['width'][j])))
                file.write("      <ymax>{}</ymax>\n".format(int(ymin + ann['height'][j])))
                file.write("    </bndbox>\n")
                file.write("  </object>\n")

            file.write("</annotation>\n")
        print(i)

        
f = h5py.File(r'data\VOCdevkit\VOC2007\PNGImages\digitStruct.mat','r') #digitStruct.mat
construct_annotation_files(f)

In [ ]:
import os
import random

trainval_percent = 0.8 #確定用於訓練的數據佔比
train_percent = 0.75 #確定在用於訓練的數據中，訓練集的佔比
xmlfilepath = r'data\VOCdevkit\VOC2007\Annotations' #將被劃分的xml文件
txtsavepath = r'data\VOCdevkit\VOC2007\ImageSets\Main' #劃分後 得到的txt保存的地方


random.seed(4)

total_xml = os.listdir(xmlfilepath)
num = len(total_xml) 
list = range(num)
tv = int(num * trainval_percent) #trainval的數目
tr = int(tv * train_percent) #train的數目
trainval = random.sample(list, tv) #被選中的文件編號
train = random.sample(trainval, tr)

ftrainval = open(os.path.join(txtsavepath,'trainval.txt'), 'w')
ftest = open(os.path.join(txtsavepath,'test.txt'), 'w')
ftrain = open(os.path.join(txtsavepath,'train.txt'), 'w')
fval = open(os.path.join(txtsavepath,'val.txt'), 'w')

for i in list:
    name = total_xml[i][:-4] + '\n' 
    if i in trainval:
        ftrainval.write(name)
        if i in train:
            ftrain.write(name)
        else:
            fval.write(name)
    else:
        ftest.write(name)

ftrainval.close()
ftrain.close()
fval.close()
ftest.close()